In [18]:
import torch.nn as nn
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

class SlidingWindowDataset(Dataset):
    def __init__(self, dataset, sequence_length):
        self.dataset = dataset
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.dataset) - self.sequence_length

    def __getitem__(self, idx):
        x, y = [], []
        for i in range(idx, idx + self.sequence_length):
            data, label = self.dataset[i]
            x.append(data)
            y.append(label)
        _, y = self.dataset[idx + self.sequence_length]
        return torch.stack(x), y


class SlidingWindowDatasetTEST(Dataset):
    def __init__(self, dataset, sequence_length):
        self.dataset = dataset
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.dataset) - self.sequence_length

    def __getitem__(self, idx):
        return self.dataset[idx:idx+self.sequence_length]
    
#Test the operational system
    
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

#read the data

df = pd.read_csv('GOOGL.csv')

features = df[['EntityCommonStockSharesOutstanding', 'AccountsPayableCurrent', 'AccountsReceivableNet', 'Assets', 'AssetsCurrent', 'CashAndCashEquivalentsAtCarryingValue', 'CostOfRevenue', 'CostsAndExpenses', 'Depreciation', 'DerivativeAssets', 'DerivativeLiabilities', 'EarningsPerShareDiluted', 'Goodwill', 'GrossProfit', 'IncomeTaxesPaidNet', 'InventoryNet', 'Liabilities', 'LiabilitiesCurrent', 'LongTermDebt', 'NetIncomeLoss', 'AssetsNoncurrent', 'OperatingIncomeLoss', 'PropertyPlantAndEquipmentNet', 'Revenues', 'ShortTermBorrowings', 'StockholdersEquity', 'TB3MS', 'DCOILWTICO', 'close', 'adjclose']]
labels = df['adjclose'].shift(-60)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

features_tensor = torch.FloatTensor(scaled_features)
labels_tensor = torch.FloatTensor(labels.to_numpy())

sequence_length = 10
dataset = list(zip(features_tensor, labels_tensor))

########################################################################################################################################################################################################################

class StockPredictor(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size=1, num_layers=2, dropout_rate=0.2):
        super(StockPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=num_layers,
                            dropout=dropout_rate, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_layer_size * 2, output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out)
        return predictions

########################################################################################################################################################################################################################

train_size = int(len(dataset) * 0.9)
train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

train_loader = DataLoader(dataset=train_dataset, batch_size=30)
test_loader = DataLoader(dataset=test_dataset, batch_size=30)

model = StockPredictor(input_size=30, hidden_layer_size=500).to(device) 
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

epochs = 1238
for epoch in range(epochs):
    model.train()
    for seq, labels in train_loader:
        seq, labels = seq.to(device), labels.unsqueeze(-1).to(device)
        optimizer.zero_grad()
        y_pred = model(seq)
        loss = loss_function(y_pred, labels)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}/{epochs}, Loss: {loss.item()}')
torch.save(model,'gaming_model.pth')
def evaluate_model(test_loader, model, loss_function, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for seq, labels in test_loader:
            seq, labels = seq.to(device), labels.unsqueeze(-1).to(device)
            predictions = model(seq)
            loss = loss_function(predictions, labels)
            test_loss += loss.item()
    
    avg_test_loss = test_loss / len(test_loader)
    print(f'Average Test Loss: {avg_test_loss:.4f}')
evaluate_model(test_loader, model, loss_function, device)



cuda
Epoch 0/1238, Loss: 18198.955078125
Epoch 0/1238, Loss: 16663.8671875
Epoch 0/1238, Loss: 12813.740234375
Epoch 0/1238, Loss: 12655.4482421875
Epoch 0/1238, Loss: 12546.7685546875
Epoch 0/1238, Loss: 13737.6611328125
Epoch 0/1238, Loss: 11099.14453125
Epoch 0/1238, Loss: 9745.1982421875
Epoch 0/1238, Loss: 8772.5234375
Epoch 0/1238, Loss: 8798.505859375
Epoch 0/1238, Loss: 8188.5576171875
Epoch 0/1238, Loss: 9479.7001953125
Epoch 0/1238, Loss: 9078.01953125
Epoch 0/1238, Loss: 10975.5244140625
Epoch 0/1238, Loss: 12642.818359375
Epoch 0/1238, Loss: 15122.3779296875
Epoch 0/1238, Loss: 14398.0380859375
Epoch 0/1238, Loss: 16456.435546875
Epoch 0/1238, Loss: 17903.255859375
Epoch 0/1238, Loss: 17869.724609375
Epoch 0/1238, Loss: 16635.40625
Epoch 0/1238, Loss: 17463.3984375
Epoch 0/1238, Loss: 18983.84765625
Epoch 0/1238, Loss: 21262.248046875
Epoch 1/1238, Loss: 17267.5859375
Epoch 1/1238, Loss: 15627.01171875
Epoch 1/1238, Loss: 11739.45703125
Epoch 1/1238, Loss: 11833.75390625
Ep

KeyboardInterrupt: 